# Дескрипторы

https://habr.com/ru/post/122082/

Чтобы понять, что такое дескриптор, обратимся к документации питона:

In general, a descriptor is an object attribute with “binding behavior”, one whose attribute access has been overridden by methods in the descriptor protocol. Those methods are `__get__()`, `__set__()`, and `__delete__()`. If any of those methods are defined for an object, it is said to be a descriptor.

### Что это означает

Дескриптор - это любой объект, который "привязывается" к объекту класса в виде атрибута это класса, при условии, если в нём определен способ этого привязывания - методы `__get__()`, `__set__()` и `__delete__()`.

### Когда это происходит

Эти методы вызываются тогда, когда мы обращаемся к дескриптору через объект класса и точку: `obj.our_descriptor_attr`.

### Нюансы

- Если определен один из перечисленных методов - объект считается дескриптором;
- если объект дескриптора определяет `__get__`, `__set__` - data дескриптором;
- если объект дескриптора определяет `__get__` - non-data дескриптором.

Они отличаются приоритетом вызова по отношению к полю `__dict__`.

## Data-дескриптор

In [ ]:
class DataDescriptor:
    def __get__(self, obj, cls):
        print("__get__")
        print(f"Вызов из объекта: {obj}")
        print(f"Класс вызывающего объекта: {cls}")
        print()

    def __set__(self, obj, val):
        print("__set__")
        print(f"Присвоение значения {val} объекту {obj}")
        print()

    def __delete__(self, obj):
        print("__del__")
        print(f"Удаляем атрибут из объекта {obj}")
        print()


class SomeData:
    data = DataDescriptor()


d = SomeData()
SomeData.data  # вот тут будет вызван __get__ с obj None
d.data
d.data = 1
del d.data

Видим, что при обращении к дескриптору срабатывают его магические методы. Однако стоит понимать, что они вызываются только тогда, когда обращение к атрибуту происходит через точку. Например, в следующем примере setter не будет вызван:

In [ ]:
d.__dict__['data'] = 1
d.data

In [ ]:
d.__getattribute__("data")

Поскольку дескриптор - это поле класса, то присвоение в это поле любого другого объекта просто удалит ссылку на дескриптор:

In [ ]:
SomeData.data = 1
print(SomeData.data)
print(d.data)
print(SomeData().data)
del SomeData.data
print(SomeData.data)

## Non-data дескриптор

In [ ]:
class NonDataDescriptor:
    def __get__(self, obj, cls):
        print(f"Вызов из объекта: {obj}")
        print(f"Класс вызывающего объекта: {cls}")
        print()


class SomeData:
    data = NonDataDescriptor()


d = SomeData()
SomeData.data  # вот тут будет вызван __get__ с obj None
d.data

print("А теперь изменим d.data. Метод __set__ дескриптора не определен, поэтому ссылка в переменной d.data перезаписалась на 1:")
d.data = 1
print(d.data)

## Резюме

- дескрипторы вызываются с помощью метода `__getattribute__`
- переопределение `__getattribute__` прекратит автоматический вызов дескрипторов
- `__getattribute__` доступен только внутри классов и объектов нового стиля
- `object.__getattribute__` и `type.__getattribute__` делают разные вызовы к `__get__`
- дескрипторы данных всегда имеют преимущество перед переменными объекта
- дескрипторы не данных могут потерять преимущество из-за переменных объекта

## Еще примеры
### Классический дескриптор-логгер

In [ ]:
class RevealAccess(object):
    """Дескриптор данных, который устанавливает и возвращает значения,
       и печатает сообщение о том, что к атрибуту был доступ.
    """

    def __init__(self, initval=None, name='атрибут'):
        self.val = initval
        self.name = name
    
    def __get__(self, obj, objtype):
        print('Получаю', self.name)
        return self.val

    def __set__(self, obj, val):
        print('Обновляю' , self.name)
        self.val = val
        

class MyClass(object):
    x = RevealAccess(10, 'атрибут "x"')
    y = 5
    
    
m = MyClass()
print(m.x)

m.x = 20
print(m.x)

print(m.y)

## Зачем нужны дескрипторы

В питоне многие вещи написаны с использованием дескрипторов, например, property. В общем случае это гибкий инструмент для работы с атрибутами класса со стороны самого атрибута, а встроенные решения предоставляют более высокоуровневый SDK к дескрипторам. Рассмотрим реализацию property:

In [ ]:

class C:
    def getx(self): return self._x

    def setx(self, value): self._x = value

    def delx(self): del self._x

    x = property(getx, setx, delx, "I'm the 'x' property.")


class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)


class CWithDescriptor:
    def getx(self): return self._x

    def setx(self, value): self._x = value

    def delx(self): del self._x

    x = Property(getx, setx, delx, "I'm the 'x' property.")


c = C()
c1 = CWithDescriptor()
c.x = 1
c1.x = 1
print('c.__dict__', c.__dict__)
print('c1.__dict__', c1.__dict__)

Staticmethod и classmethod - тоже дескрипторы.

In [ ]:
class E:
    def f(x):
        print(x)

    f = staticmethod(f)


class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f


class Analog:
    @StaticMethod
    def p():
        print("hi")
        
        
Analog.p()

In [ ]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        def newfunc(*args):
            return self.f(klass, *args)

        return newfunc
    
    
class Analog:
    @ClassMethod
    def p(cls):
        print("class is:", cls)
        
        
Analog.p()

## Привязка методов

Кстати говоря, если нам нужно вызывать один и тот же метод много раз, то быстрее будет его отвязать от объекта:

In [ ]:
class A:
    def foo(self):
        pass


a = A()
binding = a.foo

In [ ]:
%%timeit

a.foo()

In [ ]:
%%timeit

binding()

## Задание

Написать дескриптор, который будет хранить значение, но по вызову метода `null` дескриптора все значения всех инициализированных дескрипторов обнулятся.

Подсказка: для этого нужно хранить внутри дескриптора ссылки на все содержащие его классы и список названий атрибутов, в которых записана ссылка на дескриптор (названия можно получить, например, через `__dict__` объекта, сравнив класс атрибута с классом-дескриптором).